In [4]:
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from scipy.optimize import curve_fit
from uncertainties import ufloat
from uncertainties import unumpy as unp
import array_to_latex as a2l
import csv
import pandas as pd
from astropy.io.votable import parse
from astropy.table import QTable, Table, Column

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']}) #font na grafih je LaTexov
rc('text', usetex=True)

# csv datotekam odstrani vse razen ene vrstice z besedilom (seconds, volt, volt, volt)

data = pd.read_csv('meritve/CSV1/SDS00001.csv', delimiter=',')
data = np.array(data)
t = data[:, 0]

cond = -10 * 1e-6 < t

t = t[cond]

U1 = data[:, 1]
U1 = U1[cond]

U2 = data[:, 2]
U2 = U2[cond]

step = data[:, 3]


fig, axs = plt.subplots(2, 1)

axs[0].scatter(1e6 * t, 1e3 * U1, marker='o', color='k', s=8, alpha=0.5)
axs[1].scatter(1e6 * t, 1e3 * U2, marker='o', s=8, alpha=0.5)

# ind. za maksimume in minimume U1

n = 0

t_slice = 16e-6

ind_max, ind_min = [], []
slice_ind = (t > n * t_slice) * (t < (n + 1) * t_slice)
while np.any(slice_ind):
    ind_max.append(np.argmax((U1 + 10) * slice_ind))
    ind_min.append(np.argmin((U1 - 10) * slice_ind))
    n += 1
    slice_ind = (t > (n * t_slice - 0)) * (t < ((n + 1) * t_slice - 0))

axs[0].scatter(1e6 * t[ind_max], 1e3 * U1[ind_max], marker='o', color='r', s=8, alpha=0.5)
axs[0].scatter(1e6 * t[ind_min], 1e3 * U1[ind_min], marker='o', color='r', s=8, alpha=0.5)

#ind. za maksimume in minimume U2

n = 0

ind_max, ind_min = [], []
slice_ind = (t > n * t_slice) * (t < (n + 1) * t_slice)
while np.any(slice_ind):
    ind_max.append(np.argmax((U2 + 10) * slice_ind))
    ind_min.append(np.argmin((U2 - 10) * slice_ind))
    n += 1
    slice_ind = (t > (n * t_slice - 0)) * (t < ((n + 1) * t_slice - 0))

axs[1].scatter(1e6 * t[ind_max], 1e3 * U2[ind_max], marker='o', color='r', s=8, alpha=0.5)
axs[1].scatter(1e6 * t[ind_min], 1e3 * U2[ind_min], marker='o', color='r', s=8, alpha=0.5)

In [3]:
from matplotlib.ticker import AutoMinorLocator

def prvi_del(ime, C, t_slice, t0=0):

    # csv datotekam odstrani vse razen ene vrstice z besedilom (seconds, volt, volt, volt)

    data = pd.read_csv(ime, delimiter=',')
    data = np.array(data)
    t = data[:, 0]

    cond = -10 * 1e-6 < t #vse meritve večje od 10 * 1e-6

    t = t[cond]

    U1 = data[:, 1]
    U1 = U1[cond]

    U2 = data[:, 2]
    U2 = U2[cond]


    fig, axs = plt.subplots(2, 1)

    axs[0].scatter(1e6 * t, 1e3 * U1, marker='o', color='k', s=8, alpha=0.5, label=r'meritve')
    axs[1].scatter(1e6 * t, 1e3 * U2, marker='o', s=8, alpha=0.5, label=r'meritve')

    # ind. za maksimume in minimume U1

    n = 0

    ind_max, ind_min = [], []
    slice_ind = (t > n * t_slice) * (t < (n + 1) * t_slice)
    while np.any(slice_ind):
        ind_max.append(np.argmax((U1 + 10) * slice_ind))
        ind_min.append(np.argmin((U1 - 10) * slice_ind))
        n += 1
        slice_ind = (t > (n * t_slice - t0)) * (t < ((n + 1) * t_slice - t0))

    axs[0].scatter(1e6 * t[ind_max], 1e3 * U1[ind_max], marker='o', color='r', s=8, alpha=0.5)
    axs[0].scatter(1e6 * t[ind_min], 1e3 * U1[ind_min], marker='o', color='r', s=8, alpha=0.5)

    #ind. za maksimume in minimume U2

    n = 0

    ind_max, ind_min = [], []
    slice_ind = (t > n * t_slice) * (t < (n + 1) * t_slice)
    while np.any(slice_ind):
        ind_max.append(np.argmax((U2 + 10) * slice_ind))
        ind_min.append(np.argmin((U2 - 10) * slice_ind))
        n += 1
        slice_ind = (t > (n * t_slice - t0)) * (t < ((n + 1) * t_slice - t0))

    axs[1].scatter(1e6 * t[ind_max], 1e3 * U2[ind_max], marker='o', color='r', s=8, alpha=0.5)
    axs[1].scatter(1e6 * t[ind_min], 1e3 * U2[ind_min], marker='o', color='r', s=8, alpha=0.5)

    for i, ax in enumerate(axs): 
        ax.set_title(f'Potek $U_{i + 1}$ pri $C_0 = {C} pF$')
        ax.set_xlabel(r'$t [\mu s]$')
        ax.set_ylabel(r'$U [mV]$')
        ax.legend()
        ax.set_axisbelow(True)
        ax.grid()
        ax.grid(which='minor', linestyle=':')
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    fig.tight_layout()
    fig.savefig(f'C{C}.pdf')

In [4]:
#prvi_del('meritve/CSV1/SDS00001.csv', 0, 16e-6)
#prvi_del('meritve/CSV1/SDS00002.csv', 150, 16e-6)
#prvi_del('meritve/CSV1/SDS00003.csv', 330, 16e-6)
#prvi_del('meritve/CSV1/SDS00004.csv', 560, 16e-6)
#prvi_del('meritve/CSV1/SDS00005.csv', 820, 16e-6)
#prvi_del('meritve/CSV1/SDS00006.csv', 1150, 16e-6)

In [5]:
def omega (N, Nt_0, dNt_0): 
    Nt_0 = unp.uarray([Nt_0], [dNt_0])
    return 2 * np.pi * Nt_0

In [6]:
 def domega_dBeta(T_0, dT_0, U_0, dU_0, bU_T_0, dU_T_0):
    T_0 = unp.uarray([T_0], [dT_0])
    U_0 = unp.uarray([U_0], [dU_0])
    bU_T_0 = unp.uarray([bU_T_0], [dU_T_0])
    omega = 2*np.pi / T_0
    beta = 1/T_0 * unp.log(U_0 / bU_T_0)
    return omega, beta 

ERROR! Session/line number was not unique in database. History logging moved to new session 101


In [7]:
# U1 

omegaU1a = omega(14, 200e-6, 5e-6)
omegaU1b = omega(27, 400e-6, 5e-6)
omegaU1c = omega(38, 580e-6, 5e-6)
omegaU1d = omega(37, 600e-6, 5e-6)
omegaU1e = omega(37, 635e-6, 5e-6)
omegaU1f = omega(36, 630e-6, 5e-6)

# U2

omegaU2a = omega(22, 310e-6, 5e-6)
omegaU2b = omega(33, 500e-6, 5e-6)
omegaU2c = omega(40, 600e-6, 5e-6)
omegaU2d = omega(21, 310e-6, 5e-6)
omegaU2e = omega(21, 340e-6, 5e-6)
omegaU2f = omega(30, 500e-6, 5e-6)

In [7]:
# U1

domegaU1a, betaU1a = domega_dBeta(200e-6, 5e-6, 8.2e-3, 0.2e-3, 1e-3, 0.2e-3)
domegaU1b, betaU1b = domega_dBeta(260e-6, 5e-6, 8.2e-3, 0.5e-3, 1.5e-3, 0.5e-3)
domegaU1c, betaU1c = domega_dBeta(340e-6, 5e-6, 8e-3, 0.5e-3, 1e-3, 0.5e-3)
domegaU1d, betaU1d = domega_dBeta(350e-6, 5e-6, 8e-3, 0.5e-3, 1e-3, 0.5e-3)
domegaU1e, betaU1e = domega_dBeta(370e-6, 5e-6, 7.5e-3, 0.5e-3, 1e-3, 0.5e-3)
domegaU1f, betaU1f = domega_dBeta(310e-6, 5e-6, 7e-3, 0.5e-3, 1e-3, 0.5e-3)


# U2 

domegaU2a, betaU2a = domega_dBeta(280e-6, 5e-6, 2.4e-3, 0.2e-3, 0.4e-3, 0.2e-3)
domegaU2b, betaU2b = domega_dBeta(355e-6, 5e-6, 3.75e-3, 0.3e-3, 0.5e-3, 0.3e-3)
domegaU2c, betaU2c = domega_dBeta(400e-6, 5e-6, 5e-3, 0.5e-3, 0.5e-3, 0.5e-3)
domegaU2d, betaU2d = domega_dBeta(265e-6, 5e-6, 6e-3, 0.5e-3, 1e-3, 0.5e-3)
domegaU2e, betaU2e = domega_dBeta(270e-6, 5e-6, 6e-3, 0.5e-3, 1e-3, 0.5e-3)
domegaU2f, betaU2f = domega_dBeta(220e-6, 5e-6, 6e-3, 0.5e-3, 1e-3, 0.5e-3)


[31415.92653589793+/-785.3981633974482]
[10520.670771351037+/-1041.1771803420331]


(array([31415.92653589793+/-785.3981633974482], dtype=object),
 array([10520.670771351037+/-1041.1771803420331], dtype=object))

In [8]:
data = np.loadtxt('meritve/2del.dat')
print(data)

[[4.52500e+04 4.62392e-03]
 [4.55000e+04 4.67934e-03]
 [4.57500e+04 4.74664e-03]
 [4.60000e+04 4.81590e-03]
 [4.62500e+04 4.88782e-03]
 [4.65000e+04 4.95998e-03]
 [4.67500e+04 5.03075e-03]
 [4.70000e+04 5.10796e-03]
 [4.72500e+04 5.18701e-03]
 [4.75000e+04 5.26860e-03]
 [4.77500e+04 5.35068e-03]
 [4.80000e+04 5.43418e-03]
 [4.82500e+04 5.51971e-03]
 [4.85000e+04 5.60957e-03]
 [4.87500e+04 5.70065e-03]
 [4.90000e+04 5.79459e-03]
 [4.92500e+04 5.88880e-03]
 [4.95000e+04 5.98920e-03]
 [4.97500e+04 6.09152e-03]
 [5.00000e+04 6.19573e-03]
 [5.02500e+04 6.30088e-03]
 [5.05000e+04 6.40502e-03]
 [5.07500e+04 6.52337e-03]
 [5.10000e+04 6.64546e-03]
 [5.12500e+04 6.76405e-03]
 [5.15000e+04 6.89297e-03]
 [5.17500e+04 7.02031e-03]
 [5.20000e+04 7.15745e-03]
 [5.22500e+04 7.29704e-03]
 [5.25000e+04 7.44162e-03]
 [5.27500e+04 7.59027e-03]
 [5.30000e+04 7.74391e-03]
 [5.32500e+04 7.90236e-03]
 [5.35000e+04 8.06825e-03]
 [5.37500e+04 8.23893e-03]
 [5.40000e+04 8.41842e-03]
 [5.42500e+04 8.60229e-03]
 